In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import csv
from datetime import datetime
import re

In [3]:
url = "https://in.indeed.com/jobs?q={}&l={}"
r = requests.get(url)
htmlContent = r.content
#  print(htmlContent)

In [4]:
def get_url(position,location):
    url = "https://in.indeed.com/jobs?q={}&l={}"
    url1 = url.format(position,location)
    return url1
    

In [5]:
url = get_url("web developer","Navi Mumbai Maharashtra")

# Extracting raw Html

In [6]:
response = requests.get(url)

In [7]:
response

<Response [200]>

In [8]:
response.reason

''

In [9]:
soup = BeautifulSoup(response.text,'html.parser')

In [10]:
cards = soup.find_all('div','slider_container')
cards

[<div class="slider_container"><div class="slider_list"><div class="slider_item"><div class="job_seen_beacon"><table cellpadding="0" cellspacing="0" class="jobCard_mainContent" role="presentation"><tbody><tr><td class="resultContent"><div class="heading4 color-text-primary singleLineTitle tapItem-gutter"><h2 class="jobTitle jobTitle-color-purple jobTitle-newJob"><div class="new topLeft holisticNewBlue desktop"><span class="label">new</span></div><span title="Web Developer">Web Developer</span></h2></div><div class="heading6 company_location tapItem-gutter"><pre><span class="companyName">DotSyndicate</span><div class="companyLocation">Mumbai, Maharashtra</div></pre></div><div class="heading6 tapItem-gutter metadataContainer"><div class="metadata salary-snippet-container"><span aria-label="₹25,000 to ₹35,000 a month" class="salary-snippet">₹25,000 - ₹35,000 a month</span></div></div><div class="heading6 error-text tapItem-gutter"></div></td></tr></tbody></table><table class="jobCardShelf

In [11]:
len(cards)

15

# prototype the model with a single record

In [12]:
card = cards[0]
# card

In [13]:
card.div.span
# card.attrs

<span class="label">new</span>

In [14]:
card['title'] = card.find('span').get_text()

In [15]:
card['title']

'new'

In [16]:
job_title = card['title']
job_title

'new'

In [17]:
company = card.find('span','companyName').text
company

'DotSyndicate'

In [18]:
job_location= card.find('div','companyLocation').text
job_location

'Mumbai, Maharashtra'

In [26]:
job_summary = card.find('div','job-snippet').text
job_summary.strip()
job_summary

'\nRegular exposure to business stakeholders and executive management, as well as the authority and scope, to apply your expertise to many interesting technical…\n'

In [27]:
date_job = card.find('span','date').text
date_job

'7 days ago'

In [28]:
today = datetime.today().strftime('%Y-%m-%d')
today

'2021-10-04'

In [29]:
salary = card.find('span','salary-snippet').text
salary

'₹35,000 a month'

In [30]:
try:
    salary = card.find('span','salary-snippet').text
except AttributeError:
    job_salary = ' '

# Generalize the model with the function

In [31]:
def get_record(card):
    card.div.span
    card['title'] = card.find('span').get_text()
    job_title = card['title']
    company = card.find('span','companyName').text
    job_location= card.find('div','companyLocation').text
    job_summary = card.find('div','job-snippet').text
    date_job = card.find('span','date').text
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('span','salary-snippet').text
    except AttributeError:
        job_salary = ' '
    record = (job_title,company,job_location,job_summary,date_job,job_salary,today)
    return record 

In [32]:
records = []

for card in cards:
    record = get_record(card)
    records.append(record)

In [33]:
records[0]

('new',
 'Apogee Digitech Pvt Ltd',
 'Navi Mumbai, Maharashtra',
 '\nRegular exposure to business stakeholders and executive management, as well as the authority and scope, to apply your expertise to many interesting technical…\n',
 '7 days ago',
 '₹35,000 a month',
 '2021-10-04')

In [34]:
records[1]

('PHP Web Developer',
 'Beak Media',
 'Mumbai, Maharashtra•Temporarily Remote',
 '\nSalary: *Rs 35000 - Rs 50000 per month (finalized based on experience and skill sets).\nFront end knowledge under HTML, CSS, React, Bootstrap, JavaScript, MySQL…\n',
 '11 days ago',
 '₹35,000 - ₹50,000 a month',
 '2021-10-04')

# Getting the next page

In [35]:
url ='https://in.indeed.com' + soup.find('a',{'aria-label': 'Next'}).get('href')
url

'https://in.indeed.com/jobs?q=web+developer&l=Navi+Mumbai+Maharashtra&start=10'

In [36]:
i = 0 
for i in range(0,3):
    
    try:
        url ='https://in.indeed.com' + soup.find('a',{'aria-label': 'Next'}).get('href')
    except AttributeError:
        break
        
        response = requests.get(url)
        soup = BeautifulSoup(response.text,'html.parser')
        cards = soup.find_all('div','slider_container')
        
        for card in cards:
            record = get_record(card)
            records.append(record)

In [37]:
len(records)

15

# Putting it all together

In [38]:
import csv
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup

def get_url(position,location):
    url = "https://in.indeed.com/jobs?q={}&l={}"
    url1 = url.format(position,location)
    return url1

def get_record(card):
    card.div.span
    card['title'] = card.find('span').get_text()
    job_title = card['title']
    company = card.find('span','companyName').text
    job_location= card.find('div','companyLocation').text
    job_summary = card.find('div','job-snippet').text
    date_job = card.find('span','date').text
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        job_salary = card.find('span','salary-snippet').text
    except AttributeError:
        job_salary = ' '
    record = (job_title,company,job_location,job_summary,date_job,job_salary,today)
    return record

def main(position,location):
    records = []
    url = get_url(position,location)
    
    i = 0 
for i in range(0,3):
    
    try:
        url ='https://in.indeed.com' + soup.find('a',{'aria-label': 'Next'}).get('href')
    except AttributeError:
        break
        
        response = requests.get(url)
        soup = BeautifulSoup(response.text,'html.parser')
        cards = soup.find_all('div','slider_container')
        
        for card in cards:
            record = get_record(card)
            records.append(record)
    
    with open('result.csv','w',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['JobTitle','Company','Location','Summary','ExtractDate','Salary',
                         'Today_Date'])
        writer.writerows(records)

In [39]:
main('web developer','Navi Mumbai, Maharashtra')

In [40]:
import pandas as pd

In [41]:
address = 'result.csv'
data = pd.read_csv(address)
data

,JobTitle,Company,Location,Summary,ExtractDate,Salary,Today_Date
0,new,Apogee Digitech Pvt Ltd,"Navi Mumbai, Maharashtra",\nRegular exposure to business stakeholders an...,7 days ago,"₹35,000 a month",2021-10-04
1,PHP Web Developer,Beak Media,"Mumbai, Maharashtra•Temporarily Remote",\nSalary: *Rs 35000 - Rs 50000 per month (fina...,11 days ago,"₹35,000 - ₹50,000 a month",2021-10-04
2,Web Developer,Makebot Robotic Solutions Pvt. Ltd,"Navi Mumbai, Maharashtra",\nWorking closely with web designers and progr...,18 days ago,"₹20,000 - ₹25,000 a month",2021-10-04
3,new,GIT Technologies,"Vashi, Navi Mumbai, Maharashtra",\nSkills At least 1 - 2 years of experience us...,5 days ago,,2021-10-04
4,Web Developer,PTS Systems & Solutions Pvt Ltd,"Navi Mumbai, Maharashtra","\nWriting well designed, testable, efficient c...",21 days ago,"₹15,000 - ₹20,000 a month",2021-10-04
5,new,ARCITECH,"Mumbai, Maharashtra",\nAssist in troubleshooting issues on web-base...,4 days ago,"₹3,000 a month",2021-10-04
6,WEB DEVELOPER,Maxval Technologies,"CBD Belapur, Navi Mumbai, Maharashtra","\nProficient understanding of web markup, incl...",30+ days ago,,2021-10-04
7,Front-End Web Developer,TIC,"Navi Mumbai, Maharashtra","\nProficient understanding of web markup, incl...",30+ days ago,,2021-10-04
8,Web Developer,Nora Tecnoanalytics,"Navi Mumbai, Maharashtra","\nJob Type: Developers with knowledge of , HTM...",30 days ago,"₹15,000 - ₹20,000 a month",2021-10-04
9,HTML Developer,G-ORDINATEUR PVT LTD,"Navi Mumbai, Maharashtra•Temporarily Remote",\nExperience building large consumer web appli...,13 days ago,"₹1,20,000 - ₹2,40,000 a year",2021-10-04


In [100]:
# data['JobTitle']

In [99]:
# s_obj = str(data)
# s_obj

In [98]:
# type(s_obj)

In [97]:
# x = re.sub('new','Web developer',s_obj)
# print(x)

In [101]:
# data

In [102]:
# len(data['JobTitle'])

In [103]:
# data['JobTitle']

In [139]:
# import numpy as np

In [140]:
# df = np.array(data)
# df
# type(df)

In [141]:
# df[0]

In [142]:
# for i in df:
#     print(type(i))
#     print(i)

In [33]:
# for link in soup.find_all('a'):
#     print(link.get('href'))